# Forward Curves
This examples shows 
1. Generating an Access Token and using it in API Calls.
2. Calling the `market-data/reference-date/v3/forward-curve/search` endpoint to find a particular curve.
3. Calling the `market-data/forward-curve/v3/curve-code` endpoint to get assessments for a specific curve code.
4. Converting response into a DataFrame.
5. Charting the Forward Curve.

## Fetching Data

### Install Dependencies
```python
pip install requests pandas matplotlib
```

Import dependencies

In [194]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

Set `username`, `password`, and `apikey` here for simplicity

In [3]:
username = ""
password = ""
apikey = ""

```{seealso}
This `get_token` function is taken from [Generating a Token](../token.ipynb)
```

In [66]:
def get_token(username, password, apikey):
  """Get an Access Token to make API calls with."""
  body = {
    "username": username,
    "password": password
  }
  headers = {
    "appkey": apikey
  }
  try:
    r = requests.post("https://api.platts.com/auth/api", data=body, headers=headers)
    r.raise_for_status()
    return r.json()["access_token"]
  except Exception as err:
    if r.status_code >= 500:
      print(err)
    else:
      print(r.status_code, r.json())

In [14]:
def search_curves(username, password, apikey, q):
  """Search for curves based on `q`"""
  token = get_token(username, password, apikey)
  headers= {
    "appkey": apikey,
    "Authorization": f"Bearer {token}"
  }

  params = {
    "q": q
  }

  r = requests.get("https://api.platts.com/market-data/reference-data/v3/forward-curve/search", headers=headers, params=params)
  return r.json()

In [1]:
data = search_curves(username, password, apikey, "Brent Frontline****")
data['results']

NameError: name 'search_curves' is not defined

In [2]:
def get_curve_by_curvecode(username, password, apikey, curve_code):
  """fetch assessments for a specific curve"""
  token = get_token(username, password, apikey)
  headers= {
    "appkey": apikey,
    "Authorization": f"Bearer {token}"
  }

  params = {
    "filter": f'curve_code: "{curve_code}"'
  }

  r = requests.get("https://api.platts.com/market-data/forward-curve/v3/curve-code", headers=headers, params=params)
  return r.json()

In [3]:
curve_data = get_curve_by_curvecode(username, password, apikey, "CN001")
curve_data

NameError: name 'username' is not defined

In [4]:
df = pd.json_normalize(curve_data['results'], record_path=["symbol_data", "data"], 
  meta=['curve_name', 'curve_code', ['symbol_data', 'symbol'], ['symbol_data', 'derivative_position'], 
    ['symbol_data', 'contract_label'], ['symbol_data', 'derivative_maturity_frequency']])
df.columns = df.columns.str.replace("symbol_data.", "", regex=True)
df.head(20)

NameError: name 'pd' is not defined

In [5]:
filt = df['derivative_maturity_frequency'] == "Month"
filt2 = df['bate'] == "c"
chart_df = df[filt & filt2]
chart_df = chart_df.set_index(pd.PeriodIndex(chart_df['contract_label'], freq="M"))

NameError: name 'df' is not defined

In [6]:
chart_df = chart_df.sort_values(by="value")
chart_df

NameError: name 'chart_df' is not defined

In [7]:
fig, ax = plt.subplots()
chart_df['value'].plot(ax=ax, figsize=(12,6), x_compat=True, label=chart_df['curve_name'][0])
ax.set_xlabel("Contract Label")
ax.legend()
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))

NameError: name 'plt' is not defined